In [1]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 子處理程序，用來取代 os.system 的功能
import subprocess

'''
Selenium with Python 中文翻譯文檔
參考網頁：https://selenium-python-zh.readthedocs.io/en/latest/index.html
selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
Mouse Hover Action in Selenium
參考網址：https://www.toolsqa.com/selenium-webdriver/mouse-hover-action/
yt-dlp 下載影音的好工具
參考網址：https://github.com/yt-dlp/yt-dlp
'''

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options,
    service = Service(ChromeDriverManager().install())
)

# 建立儲存圖片、影片的資料夾
folderPath = 'youtube'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 放置爬取的資料
listData = []

In [37]:
#走訪頁面
def visit():
    driver.get('https://www.youtube.com/')

#輸入關鍵字
def search():
    txtInput = driver.find_element(By.CSS_SELECTOR, "input#search")
    txtInput.send_keys('老天鵝')
    
    sleep(1)
    
    txtInput.submit()
    
    sleep(1)

#篩選(可做可不做)
def filterFunc():
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'ytd-toggle-button-renderer.style-scope.ytd-search-sub-menu-renderer')
                                          )
        )
        
        driver.find_element(
            By.CSS_SELECTOR,
            'ytd-toggle-button-renderer.style-scope.ytd-search-sub-menu-renderer'
        ).click()
        
        sleep(1)
        
        driver.find_elements(By.CSS_SELECTOR,
                            'yt-formatted-string.style-scope.ytd-search-filter-renderer'
                            )[9].click()
        
        sleep(1)
        
    except TimeoutException:
        print('等待逾時')
        
#滾動頁面
def scroll():
    '''
    innerHeight => 瀏覽器內部的高度
    offset => 當前捲動的量(高度)
    count => 累計無效滾動次數
    limit => 最大無效滾動次數 
    '''
    innerHeight = 0
    offset = 0
    count = 0
    limit = 3
    
    while count <= limit:
        #取得每次移動高度=當前innerHeight:
        offset = driver.execute_script(
            'return window.document.documentElement.scrollHeight'
        )
        
        #卷軸向下移動
        driver.execute_script(f'''
            window.scrollTo({{
                top: {offset},
                behavior: 'smooth'
            }});
        ''')
        
        #等待網頁loading
        sleep(10)
        
        #取得當前innerHeight
        innerHeight = driver.execute_script(
            'return window.document.documentElement.scrollHeight'
        )
        
        #判斷
#         if offset == innerHeight:
#             count +=1
            
        if offset >= 600:
            break

#分析頁面
def parse():
    #取得主要元素集合
    ytd_video = driver.find_elements(
        By.CSS_SELECTOR,
        'ytd-video-renderer.style-scope.ytd-item-section-renderer'
    )
    
    #逐一檢視元素
    for ytd_vidoe_renderer in ytd_video:
        
        print('=' * 50)
        img = ytd_vidoe_renderer.find_element(
            By.CSS_SELECTOR,
            'ytd-thumbnail.style-scope.ytd-video-renderer img#img'
        )
        
        imgSrc = img.get_attribute('src')
        print(imgSrc)
        
        #取得超連結a
        a = ytd_vidoe_renderer.find_element(
            By.CSS_SELECTOR,
            'a#video-title'
        )
        aTitle = a.get_attribute('innerText')
        print(aTitle)
        aLink = a.get_attribute('href')
        print(aLink)
        
        #取得影音ID
        strDelimiter = ''
        if 'shorts' in aLink:
            strDelimiter = '/shorts/'
        else:
            strDelimiter = 'v='
        youtube_id = aLink.split(strDelimiter)[1]
        print(youtube_id)
        
        #將資料放入dict
        listData.append({
            'id': youtube_id,
            'titel': aTitle,
            'link': aLink,
            'img': imgSrc
        })

# 將List存成Json

def saveJson():
    with open(f'{folderPath}/youtube.json', 'w', encoding='utf-8') as file:
        file.write(json.dumps(listData, ensure_ascii=False, indent=4))
        
#關閉瀏覽器
def close():
    driver.quit()

In [38]:
if __name__ == '__main__':
    visit()
    search()
    filterFunc()
    scroll()
    parse()
    saveJson()
    driver.quit()

https://i.ytimg.com/vi/KaEvuIxqocI/hq720.jpg?sqp=-oaymwEcCOgCEMoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLABta9pmEDC7IbwROinl2eDlgGVmQ
你可以理解我現在ㄉ心情嗎｜政壇馬戲團
https://www.youtube.com/watch?v=KaEvuIxqocI
KaEvuIxqocI
https://i.ytimg.com/vi/Il6jqi6AzjM/hq720_2.jpg?sqp=-oaymwEdCNAFENAFSFXyq4qpAw8IARUAAIhCcAHAAQbQAQE=&rs=AOn4CLCIkjWOBLDPH08C5xe79xxzpvOigQ
資策會の逆襲！？｜政壇馬戲團 #職場 #論文 #博士 #高虹安 #民進黨
https://www.youtube.com/shorts/Il6jqi6AzjM
Il6jqi6AzjM
https://i.ytimg.com/vi/wX2pih61BQQ/hqdefault.jpg?sqp=-oaymwEcCOADEI4CSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLD_e-FnptgoAC_DmUORrOsH-U1XNA
老天鵝柯市長過年節目
https://www.youtube.com/watch?v=wX2pih61BQQ
wX2pih61BQQ
https://i.ytimg.com/vi/4ysFVnYEkCU/hq720.jpg?sqp=-oaymwEcCOgCEMoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLA6UmpvMh2g4gPi3tb1m9Ebj-nvCg
我的老天鵝！神人揭省錢祕訣麥當勞"5漢堡+2冷飲"只花80元 民眾傻眼：沒必要吧...│【消費報你知】20180826│三立新聞台
https://www.youtube.com/watch?v=4ysFVnYEkCU
4ysFVnYEkCU
https://i.ytimg.com/vi/fco6eJq3Nus/hq720.jpg?sqp=-oaymwEcCOgCEMoBSFXyq4qpAw4IARUAAIhCGAFwAcABB

In [43]:
'''
yt-dlp 專案
https://github.com/yt-dlp/yt-dlp

下載 yt-dlp (Linux 和 MacOS 版本，要設定「chmod +x yt-dlp」，MacOS 也要改成 yt-dlp)
- Windows: https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp.exe
- Linux: https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp
- MacOS: https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp_macos

輸出檔案名稱範例
https://github.com/yt-dlp/yt-dlp#output-template-examples
'''

#下載
def download():
    #開啟json
    with open(f'{folderPath}/youtube.json', 'r', encoding='utf-8') as file:
        strJson = file.read()
    
    #將json轉成list
    listResult = json.loads(strJson)
    
    #下載所有檔案
    for index, obj in enumerate(listResult):
        if index == 3:
            break
        
        print('=' * 50)
        print(f'正在下載: {obj["link"]}')
        
        #定義CMD指令
        cmd = [
            './yt-dlp.exe',
            obj['link'],
            '-f', 'b[ext=mp4]',
            '-o', f'{folderPath}/%(title)s.%(ect)s'
        ]
        
        # 執行指令,並回傳結果
        result = subprocess.run(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT
        )
        
        # 將回傳結果進行解碼，顯示實際執行過程的文字輸出
        output = result.stdout
        print('下載完成')
        print(output)
        
        

In [44]:
download()

正在下載: https://www.youtube.com/watch?v=KaEvuIxqocI
下載完成
b'[youtube] KaEvuIxqocI: Downloading webpage\n[youtube] KaEvuIxqocI: Downloading android player API JSON\n[info] KaEvuIxqocI: Downloading 1 format(s): 22\n[download] Destination: youtube\\\xa7A\xa5i\xa5H\xb2z\xb8\xd1\xa7\xda\xb2{\xa6b\xa3x\xa4\xdf\xb1\xa1\xb6\xdc\xa1U\xacF\xbe\xc2\xb0\xa8\xc0\xb8\xb9\xce.NA\n\r[download]   0.0% of    3.20MiB at  499.44KiB/s ETA 00:06\r[download]   0.1% of    3.20MiB at  984.35KiB/s ETA 00:03\r[download]   0.2% of    3.20MiB at    1.71MiB/s ETA 00:01\r[download]   0.5% of    3.20MiB at    2.43MiB/s ETA 00:01\r[download]   0.9% of    3.20MiB at    5.01MiB/s ETA 00:00\r[download]   1.9% of    3.20MiB at   10.19MiB/s ETA 00:00\r[download]   3.9% of    3.20MiB at   15.48MiB/s ETA 00:00\r[download]   7.8% of    3.20MiB at   19.56MiB/s ETA 00:00\r[download]  15.6% of    3.20MiB at   24.78MiB/s ETA 00:00\r[download]  31.2% of    3.20MiB at   29.02MiB/s ETA 00:00\r[download]  62.5% of    3.20MiB at   31.77M